# Exploratory Data Analysis

In [ ]:
import pandas as pd

from income_predict_d100_d400 import cleaning
from income_predict_d100_d400.robust_paths import DATA_DIR
from income_predict_d100_d400 import eda

In [ ]:
parquet_path = DATA_DIR / "census_income.parquet"

df_raw = pd.read_parquet(parquet_path)
print(df_raw.head())


In [ ]:
df_raw.info()
df_raw.describe()

In [ ]:
#1. Describe your data.
eda.get_data_description(df_raw)

In [ ]:
#2. What is the distribution of the target variable?
eda.get_target_distribution(df_raw, 'income')


In [ ]:
#3. Do we face outliers and missing values?
eda.get_outliers_summary(df_raw)

### Outlier Handling Decision

Outliers detected above (particularly in `capital-gain`, `capital-loss`, and `age`) are **retained** for the following reasons:

1. **Domain validity**: Extreme capital gains/losses represent genuine high-net-worth individuals, which is the population we're trying to identify for the >USD50K prediction task. Removing them would bias the model against detecting high earners.

2. **Model robustness**: 
   - LGBM is tree-based and inherently robust to outliers (splits on rank, not magnitude)
   - GLM uses regularisation which reduces sensitivity to extreme values

3. **Information loss**: Outliers in income-related features likely carry strong predictive signal. A person with USD99,999 capital gains is almost certainly a high earner.

**Alternative considered**: Winsorisation was considered but rejected as it would artificially cap some signals that appear most predictive of high income.

### Missing Data Imputation Strategy

The dataset contains missing values in three categorical columns: `workclass`, `occupation`, and `native_country`.

**Strategy chosen: Impute with "missing" category**

| Approach | Rationale |
|----------|-----------|
| Categorical, stays missing | Missingness may be informative e.g. someone unwilling to disclose occupation or work class could correlate with income |
| Numeric, use median | Median is robust to outliers (unlike mean). Used inside sklearn pipeline to avoid information leakage.  |

**Alternatives considered:**

| Alternative | Why rejected |
|-------------|--------------|
| Drop rows with missing values | Would lose ~7% of data; missingness may not be random, so we lose potential predictor information |
| Drop columns with missing values | Would lose whole features, again we lose potential predictor information  |

In [ ]:
# Cleaning pipeline
df_clean = cleaning.full_clean(df_raw)
df_clean.columns.tolist()

In [ ]:
#4. How do specific features correlate with the target variable?
eda.get_feature_correlations(df_clean, 'high_income')

#5. What features can we use for the specific prediction task?

### Feature Selection Summary

Based on the exploratory analysis above, the following features are selected for the prediction task:

**Numeric Features:**
| Feature | Rationale |
|---------|-----------|
| `age` | Strong correlation with income; clear pattern showing income peaks mid-career |
| `education` | Ordinal encoding preserves natural ordering; higher education correlates with higher income |
| `capital_net` | Combined capital gains/losses; strong indicator of investment income |
| `hours_per_week` | Distinguishes full-time from part-time workers, as well as workers working beyond expected full-time hours |
| `is_female` | Binary encoding of sex; captures gender pay gap effects |
| `is_white`, `is_black` | Binary encoding of race; captures demographic income disparitie; This method captures three categories of race, White, Black, and Other  |

**Categorical Features:**
| Feature | Rationale |
|---------|-----------|
| `work_class` | Employment type (private, government, self-employed) affects income distribution |
| `occupation` | Strong predictor; executive and professional roles correlate with high income |
| `relationship` | Marital/household status correlates with income |
| `native_country` | May capture geographic/economic origin effects |

**Features Excluded:**
| Feature | Reason for Exclusion |
|---------|----------------------|
| `fnlwgt` | Census sampling weight; not predictive of individual income |
| `education-num` | Redundant with ordinal `education` encoding |
| `marital_status` | Multicollinearity with `relationship` field |
| `unique_id` | Artificial identifier with no predictive value |